# Train Football Pitch Key-Point Detection Model:

---

Dataset:

<a href="https://universe.roboflow.com/private-uofaa/football-field-detection-f07vi-jufj9">
    <img src="https://app.roboflow.com/images/download-dataset-badge.svg"></img>
</a>


This notebook is based on the [How to Train YOLOv8 Object Detection on a Custom Dataset](https://github.com/roboflow/notebooks/blob/main/notebooks/train-yolov8-object-detection-on-custom-dataset.ipynb) notebook from the [roboflow/notebooks](https://github.com/roboflow/notebooks) repository.

## Configure API keys:

- Open your [`Roboflow Settings`](https://app.roboflow.com/settings/api) page. Click `Copy`. This will place your private key in the clipboard.
- In Colab, go to the left pane and click on `Secrets` (🔑). Store Roboflow API Key under the name `ROBOFLOW_API_KEY`.

## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Settings` -> `Accelorator` and set it to `GPU T4x2`.

In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Install all required dependencies:


In [ ]:
!pip install -q ultralytics roboflow

In [ ]:
from roboflow import Roboflow
from google.colab import userdata
from IPython.display import Image

## Pull the dataset upon which it will be trained and testeted:

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
ROBOFLOW_API_KEY = user_secrets.get_secret("Roboflow")
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

project = rf.workspace("private-uofaa").project("football-field-detection-f07vi-jufj9")
version = project.version(1)
dataset = version.download("yolov8")

In [ ]:
!sed -i 's|\(train: \).*|\1../train/images|' {dataset.location}/data.yaml
!sed -i 's|\(val: \).*|\1../valid/images|' {dataset.location}/data.yaml

## Custom training:

In [ ]:
%cd {HOME}

!yolo task=pose mode=train model=yolov8x-pose.pt data={dataset.location}/data.yaml batch=32 epochs=500 imgsz=640 mosaic=0.0 plots=True device=0,1

In [ ]:
!ls {HOME}/runs/pose/train/

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/pose/train5/val_batch0_pred.jpg', width=600)

## Validate custom model:

In [ ]:
%cd {HOME}

!yolo task=pose mode=val model={HOME}/runs/pose/train5/weights/best.pt data={dataset.location}/data.yaml

## Deploy model on Roboflow:

In [ ]:
project.version(dataset.version).deploy(model_type="yolov8-pose", model_path=f"{HOME}/runs/pose/train5/")